# Análise de Dados com Python e Spark

<p>
Dados dois conjuntos de dados que possuem todas as requisições HTTP para o servidor da NASA Kennedy Space Center WWW na Flórida para um período específico, você fará uma análise de dados a seguir. Os conjuntos de dados representam logs que estão em arquivos ASCII com uma linha por requisição com as seguintes colunas:

* Host fazendo a requisição. Um hostname quando possível, caso contrário o endereço de internet se o nome não puder ser identificado. 
* Timestamp no formato "DIA/MÊS/ANO:HH:MM:SS TIMEZONE"
* Requisição (entre aspas)
* Código do retorno HTTP
* Total de bytes retornados 

Download dos datasets:

- Jul 01 to Jul 31, ASCII format, 20.7 MB gzip compressed,205.2 MB: ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz

- Aug 04 to Aug 31, ASCII format, 21.8 MB gzip compressed, 167.8 MB: ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz

> Use Python com PySpark para descobrir as seguintes informações:<br />
  * Número de hosts únicos.
  * O total de erros 404.
  * Os 5 URLs que mais causaram erro 404.
  * Quantidade de erros 404 por dia.
  * O total de bytes retornados.
 </p>


In [0]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [0]:
# importação 
from pyspark.sql import SparkSession
from pyspark.sql import Row
from operator import add
import numpy as np

In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [5]:
# Clone do projeto
!git clone https://github.com/leonardoamorim/aulapython.git

Cloning into 'aulapython'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 28 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [0]:
# Extracao dos arquivos
! gunzip aulapython/NASA_access_log_Jul95.gz
! gunzip aulapython/NASA_access_log_Aug95.gz

In [0]:
# Leitura dos dados de Julho e Agosto
julho = sc.textFile('aulapython/NASA_access_log_Jul95')

agosto = sc.textFile('aulapython/NASA_access_log_Aug95')

In [0]:
# Criação das lista para cada mês
lista_julho = julho.map(lambda line: line.split()).collect()
lista_agosto = agosto.map(lambda line: line.split()).collect()

# Respostas :

### Número de hosts distintos no mês de Julho

In [9]:
# Total distinto
contagem_julho = julho.map(lambda line: line.split(' ')[0]).distinct().count()
contagem_agosto = agosto.map(lambda line: line.split(' ')[0]).distinct().count()

# Monta a mensagem 
txt = 'Os valor de hosts distintos para os meses são:\n Julho = {0} \n Agosto = {1}'

# Retorno a mensagem e nem os valores
print(txt.format(contagem_julho, contagem_agosto))

Os valor de hosts distintos para os meses são:
 Julho = 81983 
 Agosto = 75060


### O total de erros 404

In [0]:
# Funcao para verificar se em uma linha existe um codigohttp igual a 404
def codigo404(linha):
    try:
      codigohttp = linha.split(' ')[-2]
      if codigohttp == '404':
        return True
    except:
        pass
    return False

In [11]:
# O total de erros 404.
erros404_julho = julho.filter(codigo404).cache()
erros404_agosto = agosto.filter(codigo404).cache()

# Monta a mensagem 
txt = 'Erros 404 em:\n Julho: {0}\n Agosto: {1}'

# Retorno dos valores
print(txt.format(erros404_julho.count(), erros404_agosto.count()))

Erros 404 em:
 Julho: 10845
 Agosto: 10056


### Os 5 URLs que mais causaram erro 404.

In [0]:
# Funcao para retornar os top5 urls com mais erros 404
def top5_hosts404(rdd):
    hosts = rdd.map(lambda linha: linha.split('"')[1].split(' ')[1])
    counts = hosts.map(lambda hosts: (hosts, 1)).reduceByKey(add)
    top5 = counts.sortBy(lambda par: -par[1]).take(5)
    
    print('\nTop 5 urls:')
    for hosts, count in top5:
        print(hosts, count)
        
    return top5
    

### Quantidade de erros 404 por dia

In [0]:
# Função para quantidade de erros 404 por dia
def contador_lista_404(lista):

  # cria lista
  lst = []

  # percorre a lista e grava somente a data
  for i in range(len(lista)):
    try:
      if '404' in lista[i]:
        lst.append(lista[i][3].split('[')[1].split(':')[0])
    except:
      pass

  # Monta tupla com o total para cada valor
  (valores, totais) = np.unique(lst, return_counts=True)
  retorno = np.asarray((valores, totais)).T

  return retorno

In [190]:
# Retorno dos valores para o mês de julho
print(contador_lista_404(lista_julho))

[['01/Jul/1995' '316']
 ['02/Jul/1995' '291']
 ['03/Jul/1995' '474']
 ['04/Jul/1995' '359']
 ['05/Jul/1995' '497']
 ['06/Jul/1995' '641']
 ['07/Jul/1995' '570']
 ['08/Jul/1995' '302']
 ['09/Jul/1995' '349']
 ['10/Jul/1995' '398']
 ['11/Jul/1995' '471']
 ['12/Jul/1995' '471']
 ['13/Jul/1995' '532']
 ['14/Jul/1995' '413']
 ['15/Jul/1995' '254']
 ['16/Jul/1995' '257']
 ['17/Jul/1995' '406']
 ['18/Jul/1995' '465']
 ['19/Jul/1995' '639']
 ['20/Jul/1995' '428']
 ['21/Jul/1995' '334']
 ['22/Jul/1995' '192']
 ['23/Jul/1995' '233']
 ['24/Jul/1995' '328']
 ['25/Jul/1995' '462']
 ['26/Jul/1995' '336']
 ['27/Jul/1995' '336']
 ['28/Jul/1995' '94']]


In [194]:
# Retorno dos valores para o mês de agosto
print(contador_lista_404(lista_agosto))

[['01/Aug/1995' '243']
 ['03/Aug/1995' '304']
 ['04/Aug/1995' '346']
 ['05/Aug/1995' '236']
 ['06/Aug/1995' '373']
 ['07/Aug/1995' '537']
 ['08/Aug/1995' '391']
 ['09/Aug/1995' '279']
 ['10/Aug/1995' '315']
 ['11/Aug/1995' '263']
 ['12/Aug/1995' '196']
 ['13/Aug/1995' '217']
 ['14/Aug/1995' '287']
 ['15/Aug/1995' '327']
 ['16/Aug/1995' '259']
 ['17/Aug/1995' '271']
 ['18/Aug/1995' '256']
 ['19/Aug/1995' '209']
 ['20/Aug/1995' '312']
 ['21/Aug/1995' '305']
 ['22/Aug/1995' '288']
 ['23/Aug/1995' '345']
 ['24/Aug/1995' '420']
 ['25/Aug/1995' '416']
 ['26/Aug/1995' '366']
 ['27/Aug/1995' '370']
 ['28/Aug/1995' '410']
 ['29/Aug/1995' '420']
 ['30/Aug/1995' '571']
 ['31/Aug/1995' '526']]


### Quantidade total de bytes

In [0]:
# Função para soma da última coluna da lista
def sumByte(lista):

  # Criação da Lista vazia e variável
  lst = []
  soma = 0

  # Percorre a lista pelo índice e retorno o valor do último elemento da lista
  for i in range(len(lista)):
    try:
      num = int(lista[i][-1])
      soma += num
    except:
      pass

  return soma

In [13]:
# Quantidade de bytes total em Julho: 38695973491
# Quantidade de bytes total em Agosto: 26828341424 
txt = 'Quantidade de bytes para os meses:\n Julho: {0} \n Agosto: {1}'

print(txt.format(sumByte(lista_julho), sumByte(lista_agosto)))

Quantidade de bytes para os meses:
 Julho: 38695973491 
 Agosto: 26828341424
